# Module 03: Graph Theory Basics

Understanding graphs is essential for grasping how neural network connectivity patterns affect learning.

## Learning Objectives
- Understand nodes, edges, and graph representations
- Learn key graph metrics (degree, clustering, path length)
- Explore classic graph models (random, regular, scale-free)
- Introduction to the Watts-Strogatz small-world model

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

np.random.seed(42)
print("[OK] Libraries loaded")

## 1. Graph Fundamentals

A graph G = (V, E) consists of vertices (nodes) and edges (connections).

In [ ]:
# Create a simple graph
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5])
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5), (5, 1), (1, 3)])

plt.figure(figsize=(8, 6))
nx.draw(G, with_labels=True, node_color='lightblue', node_size=500, font_size=12)
plt.title('Simple Graph with 5 Nodes')
plt.show()

print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")
print(f"Degree sequence: {dict(G.degree())}")

## 2. Graph Representations

Graphs can be represented as adjacency matrices or edge lists.

In [ ]:
# Adjacency matrix representation
adj_matrix = nx.adjacency_matrix(G).toarray()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Visualize adjacency matrix
im = ax1.imshow(adj_matrix, cmap='Blues')
ax1.set_title('Adjacency Matrix')
ax1.set_xlabel('Node')
ax1.set_ylabel('Node')
plt.colorbar(im, ax=ax1)

# Draw graph
nx.draw(G, ax=ax2, with_labels=True, node_color='lightblue', node_size=500)
ax2.set_title('Graph Visualization')

plt.tight_layout()
plt.show()

print("Adjacency Matrix:")
print(adj_matrix)
print("\n[OK] A[i,j] = 1 means edge between node i and j")

## 3. Key Graph Metrics

Important metrics for understanding network structure.

In [ ]:
# Create a larger graph for analysis
G = nx.watts_strogatz_graph(n=30, k=4, p=0.3, seed=42)

# Compute metrics
avg_clustering = nx.average_clustering(G)
avg_path_length = nx.average_shortest_path_length(G)
avg_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()

print("Graph Metrics:")
print(f"  - Average Degree: {avg_degree:.2f}")
print(f"  - Average Clustering Coefficient: {avg_clustering:.4f}")
print(f"  - Average Shortest Path Length: {avg_path_length:.4f}")

# Visualize degree distribution
degrees = [d for n, d in G.degree()]
plt.figure(figsize=(8, 4))
plt.hist(degrees, bins=range(min(degrees), max(degrees) + 2), alpha=0.7, edgecolor='black')
plt.xlabel('Degree')
plt.ylabel('Count')
plt.title('Degree Distribution')
plt.show()

## 4. Classic Graph Models

Comparing different graph generation models.

In [ ]:
n = 30

# Create different graph types
random_graph = nx.erdos_renyi_graph(n, p=0.15, seed=42)
regular_graph = nx.watts_strogatz_graph(n, k=4, p=0, seed=42)  # p=0 means regular ring
small_world = nx.watts_strogatz_graph(n, k=4, p=0.3, seed=42)
scale_free = nx.barabasi_albert_graph(n, m=2, seed=42)

graphs = [
    (random_graph, 'Random (Erdos-Renyi)'),
    (regular_graph, 'Regular Ring'),
    (small_world, 'Small-World (WS)'),
    (scale_free, 'Scale-Free (BA)')
]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for ax, (G, title) in zip(axes.flat, graphs):
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, ax=ax, node_size=100, node_color='lightblue', with_labels=False)
    
    # Compute metrics
    cc = nx.average_clustering(G)
    try:
        apl = nx.average_shortest_path_length(G)
    except:
        apl = float('inf')
    
    ax.set_title(f"{title}\nCC={cc:.3f}, APL={apl:.2f}")

plt.tight_layout()
plt.show()

## 5. The Watts-Strogatz Model

Small-world networks have high clustering (like regular graphs) but short path lengths (like random graphs).

In [ ]:
# Show effect of rewiring probability beta
betas = [0, 0.01, 0.1, 0.3, 0.5, 1.0]
n, k = 100, 6

clustering_values = []
path_length_values = []

for beta in betas:
    G = nx.watts_strogatz_graph(n, k, beta, seed=42)
    clustering_values.append(nx.average_clustering(G))
    path_length_values.append(nx.average_shortest_path_length(G))

# Normalize for comparison
cc_normalized = np.array(clustering_values) / clustering_values[0]
pl_normalized = np.array(path_length_values) / path_length_values[0]

plt.figure(figsize=(10, 6))
plt.semilogx([b if b > 0 else 0.001 for b in betas], cc_normalized, 'bo-', label='Clustering (C/C0)', markersize=8)
plt.semilogx([b if b > 0 else 0.001 for b in betas], pl_normalized, 'rs-', label='Path Length (L/L0)', markersize=8)
plt.axvspan(0.01, 0.3, alpha=0.2, color='green', label='Small-World Regime')
plt.xlabel('Rewiring Probability (beta)')
plt.ylabel('Normalized Value')
plt.title('Watts-Strogatz Model: Small-World Transition')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("[OK] Small-world regime: high clustering + short paths")

## 6. Small-World Coefficient

A network is "small-world" if it has high clustering relative to random graphs, but similar path length.

In [ ]:
def small_world_coefficient(G):
    """Compute the small-world coefficient sigma.
    
    sigma = (C/C_rand) / (L/L_rand)
    sigma > 1 indicates small-world properties
    """
    n = G.number_of_nodes()
    m = G.number_of_edges()
    
    # Metrics of our graph
    C = nx.average_clustering(G)
    L = nx.average_shortest_path_length(G)
    
    # Expected values for random graph with same n and m
    p = 2 * m / (n * (n - 1))
    C_rand = p  # Expected clustering for random graph
    L_rand = np.log(n) / np.log(n * p) if p > 0 else float('inf')  # Approximate
    
    sigma = (C / C_rand) / (L / L_rand) if L_rand > 0 else 0
    return sigma, C, L, C_rand, L_rand

# Test on different graphs
print("Small-World Analysis:")
print("-" * 60)

test_graphs = [
    (nx.watts_strogatz_graph(100, 4, 0.0, seed=42), "Regular Ring"),
    (nx.watts_strogatz_graph(100, 4, 0.1, seed=42), "WS (beta=0.1)"),
    (nx.watts_strogatz_graph(100, 4, 0.3, seed=42), "WS (beta=0.3)"),
    (nx.erdos_renyi_graph(100, 0.04, seed=42), "Random (ER)"),
]

for G, name in test_graphs:
    sigma, C, L, C_rand, L_rand = small_world_coefficient(G)
    status = "[Small-World]" if sigma > 1 else ""
    print(f"{name:20} sigma={sigma:.2f} C={C:.3f} L={L:.2f} {status}")

## Summary

Key concepts covered:

1. **Graph Basics**: Nodes, edges, adjacency matrix representation
2. **Graph Metrics**: Degree, clustering coefficient, path length
3. **Graph Models**: Random, regular, small-world, scale-free
4. **Watts-Strogatz**: Interpolates between regular and random
5. **Small-World Property**: High clustering + short paths

## Why This Matters for Neural Networks

- Neural network layers can be viewed as graphs (neurons = nodes, weights = edges)
- Small-world connectivity can improve information flow
- Sparse small-world networks achieve good performance with fewer parameters

## Next Steps

- [->] Module 04: Watts-Strogatz Topology in Neural Networks
- [->] Module 05: Sparse Neural Networks